In [1]:
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from PIL import Image  

print('Python version:', sys.version)
print('OpenCV version:', cv2.__version__)
print('NumPy version: ', np.__version__)


Python version: 3.8.19 (default, Mar 20 2024, 19:58:24) 
[GCC 11.2.0]
OpenCV version: 4.9.0
NumPy version:  1.24.4


In [4]:
import torch

# Model (Choose a smaller variant for faster processing)
model = torch.hub.load("ultralytics/yolov5", "yolov5s")  # or yolov5n

# Images
img = "/scratch/jankita.scee.iitmandi/yolo/datasets/yolo_data/images/val/point_movement_video_100_10666.jpg"

# Inference
results = model(img)

# Feature tracking (assuming single object detection for simplicity)
if results.pandas().xyxy[0].shape[0] > 0:  # Check if any objects detected
  # Get first detected object (assuming you want to track one object)
  detection = results.pandas().xyxy[0].iloc[0]
  
  # Extract bounding box center coordinates (assuming format: x_min, y_min, x_max, y_max)
  center_x = int((detection["xmin"] + detection["xmax"]) / 2)
  center_y = int((detection["ymin"] + detection["ymax"]) / 2)
  
  # Print or use the center coordinates for tracking (e.g., store in a list for multiple frames)
  print(f"Center coordinates: ({center_x}, {center_y})")
else:
  print("No objects detected")


Using cache found in /home/jankita.scee.iitmandi/.cache/torch/hub/ultralytics_yolov5_master
/scratch/jankita.scee.iitmandi/miniconda3/envs/yolo/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
YOLOv5 🚀 2024-6-2 Python-3.8.19 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Center coordinates: (498, 129)


In [32]:
# Function for convolution

def linear_filter(img, kernel):
  print(img.shape)
  img_x, img_y = img.shape
  kernel_x, kernel_y = kernel.shape
  kernel_x = int(np.floor(kernel_x/2))
  kernel_y = int(np.floor(kernel_y/2))

  img_padded = np.pad(img, (kernel_x, kernel_y), 'constant', constant_values=0)
  X, Y = img_padded.shape

  conv_output = np.zeros((img_x, img_y))

  for i in range(kernel_x, X-kernel_x):
      for j in range(kernel_y, Y-kernel_y):
          window = img_padded[i-kernel_x:i+kernel_x+1, j-kernel_y:j+kernel_y+1]
          conv_output[i-kernel_x][j-kernel_y] = np.sum(window*kernel)

  return conv_output

In [93]:
def getNextPoints(im1, im2, xy, movedOutFlag):
  print("In function getNextPoints")

  xy2 = np.copy(xy).astype(float)
  im1 = im1.astype(np.float32)
  im2 = im2.astype(np.float32)

  # Gaussian kernel for smoothing
  kernel = np.array([
          [1, 4, 7, 4, 1],
          [4, 16, 26, 16, 4],
          [7, 26, 41, 26, 7],
          [4, 16, 26, 16, 4],
          [1, 4, 7, 4, 1,]], dtype=np.float32) / 273.0

  img = linear_filter(im1, kernel).astype(np.float32)
  Iy, Ix =  np.gradient(img)
  print((xy[0,0], xy[0,1]))
  # The given KLT algorithm is implemented
  for i in range(len(xy)):
    center = (int(xy[i, 0]), int(xy[i, 1]))
    print(np.shape(center))
    patch_x = cv2.getRectSubPix(Ix, (15,15), (int(xy2[i,0]), int(xy2[i,1])))
    patch_y = cv2.getRectSubPix(Iy, (15,15), (int(xy2[i,0]), int(xy2[i,1])))
    A = np.array([[np.sum(patch_x * patch_x), np.sum(patch_x * patch_y)], [np.sum(patch_x * patch_y), np.sum(patch_y * patch_y)]])

    for j in range(25):
      patch_t = cv2.getRectSubPix(im2, (15,15), (int(xy2[i,0]), int(xy2[i,1]))) - cv2.getRectSubPix(img, (15, 15), (int(xy[i,0]), int(xy[i,1])))
      B = -1* np.array([[np.sum(patch_x*patch_t)],[np.sum(patch_y*patch_t)]])
      disp = np.matmul(np.linalg.pinv(A), B)

      u = disp[0]
      v = disp[1]

      xy2[i, 0] = xy2[i, 0] + u
      xy2[i, 1] = xy2[i, 1] + v

      # Checking if the norm of (u, v) is leser than the threshold (from the textbook section - 9.1.3 Incremental refinement)
      if np.hypot(u, v) <= 0.01:
        break

  # Setting the movedOutFlag to 1 if the new pixels are out of bounds      
  if xy2[i,0] >= len(im1) or xy2[i,0] < 0 or xy2[i,1] >= len(im1[0]) or xy2[i,1] < 0:
    movedOutFlag[i] = 1

  return(xy2, movedOutFlag)

In [94]:
import cv2
import os

NUMBER_OF_FRAMES = 50  # Change this to the desired number of frames
VIDEO_PATH = "/scratch/jankita.scee.iitmandi/yolo/datasets/yolo_data/point_movement_video_1_001.mp4"  # Replace "your_video.mp4" with the path to your video
FRAMES_DIR = "/scratch/jankita.scee.iitmandi/yolo/yolov5/frames"  # Directory to save the frames

# Make directory to save frames if it doesn't exist
if not os.path.exists(FRAMES_DIR):
    os.makedirs(FRAMES_DIR)

def saveFramesFromVideo(video_path, frames_dir, num_frames):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_path = os.path.join(frames_dir, f"frame_{frame_count:05d}.png")
        cv2.imwrite(frame_path, frame)

        frame_count += 1
        if frame_count >= num_frames:
            break

    cap.release()

    return frame_count

DISPLAY_RADIUS = 3
GREEN = (0, 255, 0)
YELLOW = (0, 255, 255)

In [88]:
def mainFunction():
    print("In mainFunction")
    num_frames_saved = saveFramesFromVideo(VIDEO_PATH, FRAMES_DIR, NUMBER_OF_FRAMES)
    print(f"Number of frames saved: {num_frames_saved}")
    allImgs, allPaths = readImages(FRAMES_DIR, num_frames_saved)
    print(f'number of images that were read = {len(allImgs)}')

    image0 = allPaths[0]
    xy = getKeypoints(image0)
    print("THIS IS IT", xy[0,0], xy[0,1])
    if xy is None:
        print("no points to track!")
        return
    print(f'number of detected keypoints = {xy.shape[0]}')

    image0 = cv2.imread(image0, cv2.IMREAD_GRAYSCALE)
    # display keypoints for image 0
    image0color = cv2.cvtColor(image0, cv2.COLOR_GRAY2BGR)
    corners = np.intp(np.round(xy))   
    for i in corners:
        x, y = i.ravel()
        cv2.circle(image0color, (x, y), DISPLAY_RADIUS, GREEN)

    # track the initial keypoints through all remaining images
    xyt = trackPoints(xy, allImgs)

    # in image 0, draw the paths taken by the keypoints 
    drawPaths(image0color, xyt)
    return


import os

def readImages(frames_dir, num_frames):
    print("In function readImages")
    allImages = []
    allPaths = []
    for i in range(num_frames):
        print(f'reading image {i:05d}')
        image_path = os.path.join(frames_dir, f"frame_{i:05d}.png")
        imagetmp = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        allImages.append(imagetmp)
        allPaths.append(image_path)
    return allImages, allPaths


def getKeypoints(im0):
    # Assuming keypoint_detector function takes image path and returns keypoints
    results = model(im0)

    if results.pandas().xyxy[0].shape[0] > 0:  # Check if any objects detected
    # Get first detected object (assuming you want to track one object)
        detection = results.pandas().xyxy[0].iloc[0]
        
        # Extract bounding box center coordinates (assuming format: x_min, y_min, x_max, y_max)
        center_x = int((detection["xmin"] + detection["xmax"]) / 2)
        center_y = int((detection["ymin"] + detection["ymax"]) / 2)
    
        return np.asarray([[center_x, center_y]])
    
    else:
        print("No objects detected")

def trackPoints(xy, imageSequence):
    print("In function trackPoints")
    print(f'length of imageSequence = {len(imageSequence)}')
    movedOutFlag = np.zeros(xy.shape[0])
    xyt = []

    for t in range(0, len(imageSequence)-1): # predict for all images except first in sequence
        print(f't = {t}; predicting for t = {t+1}') 
        xy2, movedOutFlag = getNextPoints(imageSequence[t], imageSequence[t+1], xy, movedOutFlag)
        xy = xy2

        for pt in xy2:
            xyt.append(pt)

        # for selected instants in time, display the latest image with highlighted keypoints 
        if t in [0, 10, 20, 30, 40, 49]:
            im2color = cv2.cvtColor(imageSequence[t+1], cv2.COLOR_GRAY2BGR)
            corners = np.intp(np.round(xy2))
             
            for c in range(0, corners.shape[0]):
                if not movedOutFlag[c]:
                    x = corners[c][0]
                    y = corners[c][1]
                    cv2.circle(im2color, (x, y), DISPLAY_RADIUS, GREEN)
            cv2.imshow("Tracked Keypoints", im2color)
            cv2.waitKey(0)
     
    return xyt

def drawPaths(im0color, xyt):
    print("In function drawPaths")

    # Using cv2.circle to draw dots for all new points in xyt, by setting radius to 0
    for pt in xyt:
        im0color = cv2.circle(im0color, (round(pt[0]),round(pt[1])), radius=0, color=YELLOW, thickness=1)

    print("FINISHED: here are the paths of the tracked keypoints")
    cv2.imshow("Tracked Keypoints", im0color)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [95]:
mainFunction()

In mainFunction
Number of frames saved: 50
In function readImages
reading image 00000
reading image 00001
reading image 00002
reading image 00003
reading image 00004
reading image 00005
reading image 00006
reading image 00007
reading image 00008
reading image 00009
reading image 00010
reading image 00011
reading image 00012
reading image 00013
reading image 00014
reading image 00015
reading image 00016
reading image 00017
reading image 00018
reading image 00019
reading image 00020
reading image 00021
reading image 00022
reading image 00023
reading image 00024
reading image 00025
reading image 00026
reading image 00027
reading image 00028
reading image 00029
reading image 00030
reading image 00031
reading image 00032
reading image 00033
reading image 00034
reading image 00035
reading image 00036
reading image 00037
reading image 00038
reading image 00039
reading image 00040
reading image 00041
reading image 00042
reading image 00043
reading image 00044
reading image 00045
reading image 

: 

: 

: 

In [68]:
print(np.asarray([[2,3], [3,5]])[0,1])

3


In [ ]:
import cv2

NUMBER_OF_FRAMES = 50  # Adjust this based on the desired number of frames to process
DISPLAY_RADIUS = 3
GREEN = (0, 255, 0)
YELLOW = (0, 255, 255)


def mainFunction(video_path):
    """
    Performs KLT tracking on a video.

    Args:
        video_path (str): Path to the video file.
    """

    print("In mainFunction")

    # Read video capture object
    cap = cv2.VideoCapture(video_path)

    # Check if video is opened successfully
    if not cap.isOpened():
        print("Error opening video!")
        return

    # List to store extracted frames
    all_frames = []

    # Read frames until a certain number of frames are captured or video ends
    for i in range(NUMBER_OF_FRAMES):
        ret, frame = cap.read()

        # Check if frame is read correctly
        if not ret:
            print(f"Failed to read frame {i}")
            break

        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Add grayscale frame to the list
        all_frames.append(gray_frame)

    # Release video capture object
    cap.release()

    print(f'number of frames read = {len(all_frames)}')

    # Process the frames as usual
    if len(all_frames) > 0:
        # Get initial keypoints from the first frame
        image0 = all_frames[0]
        xy = getKeypoints(image0)
        if xy is None:
            print("No points to track!")
            return
        print(f'number of detected keypoints = {xy.shape[0]}')

        # Track keypoints through all frames
        xyt = trackPoints(xy, all_frames)

        # Display paths on the first frame
        image0color = cv2.cvtColor(image0, cv2.COLOR_GRAY2BGR)
        drawPaths(image0color, xyt)

    else:
        print("No frames captured from the video!")


def getKeypoints(im0):
    # Implement your keypoint detector function here
    # This example uses Shi-Tomasi corners (you might need to install OpenCV contrib)
    corners = cv2.goodFeaturesToTrack(im0, maxCorners=200, qualityLevel=0.01, minDistance=10)
    return corners


def trackPoints(xy, image_sequence):
    # Implement your KLT tracking logic here
    # This is a placeholder, you'll need to fill in the tracking functionality

    xyt = []  # Initialize list to store tracked keypoints
    moved_out_flag = np.zeros(xy.shape[0])

    for t in range(len(image_sequence) - 1):
        # Implement KLT tracking to get new keypoint positions in the next frame

        # Example (replace with your actual KLT tracking):
        xy2, moved_out_flag = cv2.calcOpticalFlowPyFarneback(
            image_sequence[t], image_sequence[t + 1], xy, None, 0.5, 3, 15, 3.5, 1.2, 0
        )

        # Update tracked keypoints and flags
        xy = xy2
        for pt in xy2:
            xyt.append(pt)

    return xyt


def drawPaths(im0color, xyt):
    # Draw paths on the first frame

    for pt in xyt:
        cv2.circle(im0color, (round(pt[0]), round(pt[1])), radius=0, color=YELLOW, thickness=1)

    print("FINISHED: Here are the paths of the tracked keypoints")
    cv2.imshow("Tracked Keypoints", im0color)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Replace 'your_video.mp4' with the actual path to your video file
video_path = "your_video.mp4"
mainFunction(video_path)
